In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms.v2 as tfs
import torch.nn as nn
from tqdm import tqdm
import json
import os
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


class SunDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path , "train" if train else "test")
        self.transform = transform

        # Loading json file
        with open(os.path.join(self.path, "format.json"), 'r') as fp:
            self.format = json.load(fp)
        
        self.length = len(self.format)
        self.files = tuple(self.format.keys())
        self.targets = tuple(self.format.values())

    def __getitem__(self, item):
        path_file = os.path.join(self.path, self.files[item])
        img = Image.open(path_file).convert('RGB')

        if self.transform:
            img = self.transform(img)
        
        return img, torch.tensor(self.targets[item], dtype=torch.float32)
        
    def __len__(self):
        return self.length

# Creation of object data loader
transforms = tfs.Compose([tfs.ToImage(), tfs.ToDtype(torch.float32, scale=True)])
d_train = SunDataset("datasets\dataset_reg", transform=transforms)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)


model = nn.Sequential(
    nn.Conv2d(3, 32, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2), # (batch, 32, 128, 128)
    nn.Conv2d(32, 8, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2), # (batch, 8,64,64)
    nn.Conv2d(8, 4, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2), # (batch, 4,32,32)
    nn.Flatten(), # (batch, 4096)
    nn.Linear(4096, 128),
    nn.ReLU(),
    nn.Linear(128,2)
)

optimizer = optim.Adam(params=model.parameters(), lr= 0.001, weight_decay=0.001)
loss_function= nn.MSELoss()

epochs = 5
model.to(device)
model.train()

for _e in range(epochs):
    loss_mean = 0
    lm_count = 0

    train_tqdm = tqdm(train_data, leave=True)
    for x_train, y_train in train_tqdm:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        predict = model(x_train)
        loss = loss_function(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count+=1
        loss_mean = 1/lm_count *loss.item() + (1 -1/lm_count)*loss_mean
        train_tqdm.set_description(f"Epoch {_e+1}/epochs{epochs}, loss_mean = {loss_mean:.3f}")


st = model.state_dict()
torch.save(st, 'model_sun.tar')

# Evaluation
d_test = SunDataset("datasets\dataset_reg",train=False, transform=transforms)
test_data = data.DataLoader(d_test, batch_size = 50, shuffle = False)

Q = 0 
count = 0
model.eval()

test_tqdm = tqdm(test_data, leave=True)
for x_test, y_test in test_tqdm:
    with torch.no_grad():
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        p = model(x_test)
        Q += loss_function(p, y_test).item()
        count+=1
Q /= count
print(Q)


<>:44: SyntaxWarning: invalid escape sequence '\d'
<>:95: SyntaxWarning: invalid escape sequence '\d'
<>:44: SyntaxWarning: invalid escape sequence '\d'
<>:95: SyntaxWarning: invalid escape sequence '\d'
C:\Users\pantu\AppData\Local\Temp\ipykernel_32716\3204736435.py:44: SyntaxWarning: invalid escape sequence '\d'
  d_train = SunDataset("datasets\dataset_reg", transform=transforms)
C:\Users\pantu\AppData\Local\Temp\ipykernel_32716\3204736435.py:95: SyntaxWarning: invalid escape sequence '\d'
  d_test = SunDataset("datasets\dataset_reg",train=False, transform=transforms)


Using device: cuda


100%|██████████| 20/20 [00:01<00:00, 11.69it/s]

19121.8736328125


In [ ]:
# Inference
